<details>
<summary style="
    cursor:pointer;background:#f7f7fb;border:2px solid #297be7ff;
padding:10px 12px;border-radius:10px;font-weight:700;">
tools
</summary>

<details>
<summary style="
    cursor:pointer;background:#f7f7fb;border:2px solid #297be7ff;
padding:10px 12px;border-radius:10px;font-weight:700;">
How to bring in your tools from your toolbox
</summary>


</details>

<details>
<summary style="
    cursor:pointer;background:#f7f7fb;border:2px solid #297be7ff;
padding:10px 12px;border-radius:10px;font-weight:700;">
Where to keep your tools
</summary>


Short version: treat your tools like a **real dependency**, not copy-paste clutter.

Here are a few solid patterns, from easiest to more “engineered”:

---

## 1. The simple but solid way: a “toolbox” repo you `pip install -e`

**When:** You mostly use Python and you’re okay keeping your toolbox repo next to your project.

**Setup once:**

1. Put your reusable stuff in its own repo, e.g.:

   ```
   tools-lib/
     pyproject.toml  (or setup.py)
     tools_lib/
       __init__.py
       io_helpers.py
       eda_utils.py
       ...
   ```

2. Make it a package (basic `pyproject.toml` or `setup.py`).

**In a new project:**

```bash
your-project/
tools-lib/        # cloned next to it
```

Then in your project’s venv:

```bash
pip install -e ../tools-lib
```

Now you can:

```python
from tools_lib.io_helpers import load_parquet_safely
```

**Why this is nice:**

* You get **one canonical copy** of each tool.
* Bug fixes in `tools-lib` automatically apply to all projects once you bump the version / reinstall.
* `-e` (editable) means you can improve tools while working on any project.

---

## 2. Template repo / “starter kit” for new projects

**When:** You want each new project to start with the same skeleton (structure + some tools), but tools can diverge per project.

1. Create a repo like `project-template/`:

   ```
   project-template/
     src/
       common_tools/
         __init__.py
         logging_setup.py
         paths.py
     config/
     notebooks/
     tests/
   ```

2. When starting a new project, you either:

   * Click “Use this template” on GitHub **or**
   * Copy the folder as your starting point.

3. Refine / specialize tools directly inside the project.

**Good when:**

* You care more about fast start than centralizing every utility.
* Each project might tweak the tools heavily.

---

## 3. Git submodule / subtree: share code without a package manager

**When:** Multiple repos, maybe multiple languages, but you still want a shared code folder.

### Submodule (more explicit, a bit fiddly)

In the new project:

```bash
git submodule add https://github.com/you/tools-lib.git tools_lib
```

Project tree:

```text
your-project/
  tools_lib/   # read-only-ish copy from separate repo
  src/
  ...
```

Then import as a local module (e.g. with `src/tools_lib` on PYTHONPATH).

### Subtree (acts like you copied it, but can sync later)

```bash
git subtree add --prefix tools_lib https://github.com/you/tools-lib.git main --squash
```

You get the code as if it were part of your repo, and you *can* pull updates from the original.

---

## 4. “Real dependency” with versioning (internal PyPI / package registry)

**When:** You want this to feel like any third-party library.

1. Turn your toolbox repo into a proper package and version it: `tools-lib==0.3.0`.

2. Publish it to:

   * Private PyPI, Artifactory, or GitHub Packages **or**
   * Install directly from Git:

     ```bash
     pip install "git+https://github.com/you/tools-lib.git@v0.3.0"
     ```

3. Add to `requirements.txt` or `pyproject.toml`.

**Benefits:**

* Clear versioning: project A might use `tools-lib==0.2.1`, project B `0.4.0`.
* Updating tools is a standard “bump version → upgrade dependency” flow.

---

## 5. What *not* to do (except as a quick hack)

* **Copy-pasting files** (`utils.py` copied into every project):

  * You will lose track of which project has the “latest” fix.
  * Bug fixes become whack-a-mole.

Okay as a one-off experiment, but not a long-term strategy.

---

## How to decide (practical rule of thumb)

* If you’re **still figuring out what belongs in the toolbox**
  → Use **#1 editable toolbox repo** (fast to iterate, still centralized).

* If a project needs a very custom flavor and you don’t care about reusing those changes
  → Start from a **template repo** (#2) and edit locally.

* Once some tools feel **stable and broadly useful**
  → Promote them to a **versioned package** (#4).

If you tell me what’s in your toolbox (Python utilities? SQL snippets? config helpers?), I can sketch a tiny layout + example imports tailored to how your Telco + EDGAR projects are structured.
